In [1]:
#HIDDEN
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pickle
import ipywidgets as widgets
import ipywidgets
from matplotlib import rc
from matplotlib.legend_handler import HandlerPatch

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

# params= {'text.latex.preamble' : [r'\usepackage{amsmath}']}
# plt.rcParams.update(params)

In [2]:
#HIDDEN
def add_corner_arc(ax, line, radius=.7, color=None, text=None, text_radius=.5, text_rotatation=0, **kwargs):
    '''
    Draws an arc for p0p1p2 angle.

    Parameters
    ----------
    ax: Axis to add arc to
    line: Matplotlib line consisting of 3 points of the corner
    radius: radius to add arc
    color: color of the arc
    text: text to show on corner
    text_radius: radius to add text
    text_rotatation: extra rotation for text
    kwargs: other arguments to pass to Arc

    Returns
    -------
    arc :  Matplotlib Arc object

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    Inputs:

    '''

    lxy = line.get_xydata()

    if len(lxy) < 3:
        raise ValueError('at least 3 points in line must be available')

    p0 = lxy[0]
    p1 = lxy[1]
    p2 = lxy[2]

    width = np.ptp([p0[0], p1[0], p2[0]])
    height = np.ptp([p0[1], p1[1], p2[1]])

    n = np.array([width, height]) * 1.0
    p0_ = (p0 - p1) / n
    p1_ = (p1 - p1)
    p2_ = (p2 - p1) / n 

    theta0 = -get_angle(p0_, p1_)
    theta1 = -get_angle(p2_, p1_)

    if color is None:
        # Uses the color line if color parameter is not passed.
        color = line.get_color() 
    arc = ax.add_patch(mpl.patches.Arc(p1, width * radius, height * radius, 0, theta0, theta1, color=color, **kwargs))

    if text:
        v = p2_ / np.linalg.norm(p2_)
        if theta0 < 0:
            theta0 = theta0 + 360
        if theta1 < 0:
            theta1 = theta1 + 360
        theta = (theta0 - theta1) / 2 + text_rotatation
        pt = np.dot(rotation_transform(theta), v[:,None]).T * n * text_radius
        pt = pt + p1
        pt = pt.squeeze()
        ax.text(pt[0], pt[1], text,
                horizontalalignment='left',
                verticalalignment='top',)

    return arc


def get_angle(p0, p1=np.array([0,0]), p2=None):
    '''
    Compute angle (in degrees) for p0p1p2 corner.

    Parameters
    ----------
    p0,p1,p2: Points in the form of [x,y]

    Returns
    -------
    angle :  angle in degrees

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    Inputs:
    '''
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)


def rotation_transform(theta):
    '''
    Rotation matrix given theta.

    Parameters
    ----------
    theta: theta (in degrees)

    Returns
    -------
    A :  Rotation matrix for a given theta

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    '''
    theta = np.radians(theta)
    A = [[np.math.cos(theta), -np.math.sin(theta)],
         [np.math.sin(theta), np.math.cos(theta)]]
    return np.array(A)


# Class to create the legend handle for the 2theta angle
class HandlerWedgeShifted(HandlerPatch):
        def __init__(self, shift=0.3, **kwargs):
            self.shift = shift
            super().__init__(**kwargs)

        def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
            # Create the wedge patch
            center = width / 2, height / 2
            p = mpl.patches.Wedge(center, width / 2, orig_handle.theta1, orig_handle.theta2,
                              facecolor=orig_handle.get_facecolor(), edgecolor=orig_handle.get_edgecolor())
            self.update_prop(p, orig_handle, legend)

            # Apply the translation to the wedge
            trans_offset = mpl.transforms.Affine2D().translate(-self.shift * width, 0)
            p.set_transform(trans + trans_offset)

            return [p]


def mohr_axis(ax, min, max, step):
    '''
    Function to draw the axis for the Mohr diagram.

    Parameters
    ----------
    ax: axis object
    min: minimum value for the axis tick
    max: maximum value for the axis tick
    step: The difference between successive ticks

    Returns
    -------
    None.

    Notes
    -----
    None.
    '''
    ax.axhline(0, c='k')
    ax.axvline(0, c='k')

    ax.plot((0.985), (0), ls="", marker=">", ms=6, color="k",
                transform=ax.get_yaxis_transform(), clip_on=False)
    ax.plot((0), (0.985), ls="", marker="^", ms=6, color="k",
                transform=ax.get_xaxis_transform(), clip_on=False)
    ax.plot((0.015), (0), ls="", marker="<", ms=6, color="k",
                transform=ax.get_yaxis_transform(), clip_on=False)
    ax.plot((0), (0.015), ls="", marker="v", ms=6, color="k",
                transform=ax.get_xaxis_transform(), clip_on=False)

    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_visible(False)
    ticks = np.arange(min, max+1, step)
    updated_ticks = np.hstack([ticks[:ticks.shape[0]//2], ticks[ticks.shape[0]//2+1:]])

    ax.set_xticks(updated_ticks[1:-1])
    ax.set_yticks(updated_ticks[1:-1])

    ax.text(0.465, 0.455, '0', transform=ax.transAxes,)

    ax.text(1.015, 0.5125, r'$\sigma_\theta$', fontsize=14, transform=ax.transAxes,)
    ax.text(-0.13, 0.5125, r'$-\sigma_\theta$', fontsize=14, transform=ax.transAxes,)
    ax.text(0.5125, 1.015, r'$\tau_\theta$', fontsize=14, transform=ax.transAxes,)
    ax.text(0.5125, -0.04, r'$-\tau_\theta$', fontsize=14, transform=ax.transAxes,)
    ax.text(1.05, 0.4525, r'(Tração)', ha='center', fontsize=8, transform=ax.transAxes,)
    ax.text(-0.05, 0.4525, r'(Compressão)', ha='center', fontsize=8, transform=ax.transAxes,)
    ax.tick_params(axis='both', which='both', length=0)
    ax.text(0.5725, 1.015, r'(Sentido Horário)', fontsize=8, transform=ax.transAxes,)
    ax.text(0.6225, -0.04, r'(Sentido Anti-Horário)', fontsize=8, transform=ax.transAxes,)

def draw_rotated_square(ax, corner, size, angle):
    '''
    Function to draw the filled rotated square.

    Parameters
    ----------
    ax: axis object
    corner: left bottom corner of the square
    size: size of the sides
    angle: angle of rotation

    Returns
    -------
    None.

    Notes
    -----
    None.
    '''
    square = mpl.patches.Rectangle(corner, size, size, angle=angle, edgecolor='purple', facecolor='purple', zorder=1000)
    ax.add_patch(square)

def draw_colored_rotated_square(ax, corner, size, angle):
    '''
    Function to draw the rotated square with edges of different colors.

    Parameters
    ----------
    ax: axis object
    corner: left bottom corner of the square
    size: size of the sides
    angle: angle of rotation

    Returns
    -------
    None.

    Notes
    -----
    None.
    '''
    # Define the four corners of the square before rotation
    corners = np.array([
        (corner[0] + size, corner[1]),
        (corner[0] + size, corner[1] + size),
        (corner[0], corner[1] + size),
        corner
    ])

    # Rotate the corners around the first corner
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    rotated_corners = np.dot(corners - corner, rotation_matrix) + corner

    # Plot the sides with different colors
    colors = ['black', 'y', 'black', 'y']
    for i in range(4):
        start_point = rotated_corners[i]
        end_point = rotated_corners[(i + 1) % 4]
        ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color=colors[i], zorder=1000)

def draw_colored_rotated_square_centered(ax, center, size, angle, zorder=999):
    '''
    Function to draw the centered rotated square with edges of different colors.

    Parameters
    ----------
    ax: axis object
    corner: left bottom corner of the square
    size: size of the sides
    angle: angle of rotation

    Returns
    -------
    None.

    Notes
    -----
    None.
    '''
    # Calculate half size for easier manipulation
    half_size = size / 2
    # Define the four corners of the square before rotation around the center
    corners = np.array([
        (center[0] - half_size, center[1] - half_size),
        (center[0] + half_size, center[1] - half_size),
        (center[0] + half_size, center[1] + half_size),
        (center[0] - half_size, center[1] + half_size)
    ])


    # Rotate the corners around the center
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    rotated_corners = np.dot(corners - center, rotation_matrix) + center

    # Plot the sides with different colors
    polygon = mpl.patches.Polygon(rotated_corners, closed=True, edgecolor='none', facecolor='purple', zorder=zorder)
    ax.add_patch(polygon)
    colors = ['y', 'black', 'y', 'black']
    for i in range(4):
        start_point = rotated_corners[i]
        end_point = rotated_corners[(i + 1) % 4]
        ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color=colors[i], zorder=1000)

In [9]:
#HIDDEN
def mohr_circle(sigma_x, sigma_y, tau_xy):
    '''
    Function to draw the interactive widget.

    Parameters
    ----------
    sigma_x : Stress in the x direction
    sigma_y : Stress in the y direction
    tau_xy : Shear stress
    Returns
    -------
    None.

    Notes
    -----
    None.
    '''

    # Create figure and the 2 subplots
    fig, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

    # Add a numerical tolerance to avoid zeros in the denominator
    sigma_x += 1e-12
    sigma_y += 1e-12
    tau_xy += 1e-12

    # Calculate the necessary points for drawing the Mohr circle
    sigma_avg = (sigma_x + sigma_y) / 2  # Center of the Mohr circle
    radius = np.sqrt(((sigma_x - sigma_y) / 2)**2 + tau_xy**2)  # Radius of the Mohr circle
    sigma_1 = sigma_avg + radius  # Principal stress 1
    sigma_2 = sigma_avg - radius  # Principal stress 2
    tau_xy_max = radius  # Maximum shear stress
    sig_max = np.max(np.abs([sigma_1, sigma_2]))  # Maximum principal stress

    # Generate points on Mohr's circle
    theta = np.linspace(0, 2 * np.pi, 100)  # Angles
    x = sigma_avg + radius * np.cos(theta)  # x-coordinates of the Mohr circle
    y = radius * np.sin(theta)  # y-coordinates of the Mohr circle

    # Adding points of the Mohr circle
    h1 = axs[1].plot(sigma_1, 0, 's', markersize=4, c='navy', zorder=1000, label=r'$\sigma_{1}$')
    h2 = axs[1].plot(sigma_2, 0, 's', markersize=4, c='dodgerblue', zorder=1000, label=r'$\sigma_{2}$')
    h3 = axs[1].plot(sigma_avg, radius, 'o', markersize=4, c='orangered', zorder=1000, label=r'$\tau_{xy}^{H}$')
    h4 = axs[1].plot(sigma_avg, -radius, 'o', markersize=4, c='coral', zorder=1000, label=r'$\tau_{xy}^{AH}$')
    h6 = axs[1].plot([sigma_x, sigma_y], [-tau_xy, tau_xy], '-', c='dimgray', markersize=4, zorder=1000)
    h7 = axs[1].plot([sigma_y], [tau_xy], '^', markeredgecolor='k', c='dimgray', markersize=6, zorder=1000, label=r'$(\sigma_y, \tau_{xy})$')
    h8 = axs[1].plot([sigma_x], [-tau_xy], 'v', markeredgecolor='k', c='dimgray', markersize=6, zorder=1000, label=r'$(\sigma_x, -\tau_{xy})$')
    h9 = axs[1].plot(sigma_avg, 0, 'x', markersize=4, c='k', zorder=1000, label=r'$\frac{\sigma_x + \sigma_y}{2}$')

    # Plotting the Mohr circle
    h5 = axs[1].plot(x, y, zorder=10)

    # Trick to define the angles correctly
    theta_p_orig = np.arctan(tau_xy/((sigma_x - sigma_y) / 2 + 1e-6)) / 2
    if theta_p_orig < 0:
        theta_p = 2 * theta_p_orig
        start_angle = 0
        end_angle = -theta_p
    else:
        theta_p = 2 * np.pi - 2 * theta_p_orig
        start_angle = theta_p
        end_angle = 0

    padding = 0.02  # Padding as a fraction of the axis width
    wed = mpl.patches.Wedge((sigma_avg, 0), radius * 0.4, np.rad2deg(start_angle), np.rad2deg(end_angle), ec='k', fc='goldenrod', zorder=2, label=r'$2\theta$')
    axs[1].add_patch(wed)


    legend = axs[1].legend(fontsize=8, ncol=4, handletextpad=0.1, loc='lower center', bbox_to_anchor=(0.5, 1.05),
                       handles=[h1[0], h2[0], h3[0], h4[0], h7[0], h8[0], h9[0], wed],
                       labels=[r'$\sigma_{1}$', r'$\sigma_{2}$', r'$\tau_{xy}^{H}$',
                               r'$\tau_{xy}^{AH}$', r'$(\sigma_y, \tau_{xy})$',
                               r'$(\sigma_x, -\tau_{xy})$', r'$\frac{\sigma_x + \sigma_y}{2}$',
                               r'$2\theta = $' + f'${round(np.rad2deg(2*theta_p_orig))}$°'],
                       handler_map={mpl.patches.Wedge: HandlerWedgeShifted(shift=0.6)})

    h10 = axs[1].plot([sigma_y, sigma_x], [tau_xy, tau_xy], ls='--', alpha=0.4, lw=0.7, c='k', zorder=100)
    h11 = axs[1].plot([sigma_x, sigma_x], [-tau_xy, tau_xy], ls='--', alpha=0.4, lw=0.7, c='k', zorder=100)
    h12 = axs[1].plot([sigma_x], [tau_xy], marker='o', c='k', zorder=10000)
    h13 = axs[1].plot([sigma_2, sigma_x], [0, tau_xy], ls=':', c='orchid', zorder=100)
    h14 = axs[1].plot([sigma_x, sigma_1], [tau_xy, 0], ls=':', c='orchid', zorder=100)
    intersect_x = sigma_x
    intersect_y = tau_xy

    theta_square = theta_p
    if (intersect_y < 0):
        theta_square =  -np.pi - theta_p
    else:
        theta_square = theta_p
    if (sigma_x < sigma_y):
        theta_square =  np.pi - theta_p

    if (sigma_x < sigma_y) and (intersect_y < 0):
        theta_square = 4/2 * np.pi + theta_p

    angle = np.degrees(-theta_square / 2)


    draw_colored_rotated_square_centered(axs[0], (0, 0),  0.3, -angle, zorder=-1)  # 20 is the side length of the square
    ax2 = axs[0]
    ax2.set_aspect('equal')
    ax2.set_xlim(-0.8, 0.8)
    ax2.set_ylim(-0.8, 0.8)
    theta = -np.degrees(-theta_square / 2)

    if np.abs(sigma_y) > 1e-8:
        ax2.text(-0.06, 0.55, r'$\sigma_y$', va='center', ha='center', color='dimgray')
        ax2.text(0.06, -0.55, r'$\sigma_y$', va='center', ha='center', color='dimgray')
    if np.abs(sigma_x) > 1e-8:
        ax2.text(0.6, -0.07, r'$\sigma_x$', va='center', ha='center', color='dimgray')
        ax2.text(-0.6, -0.07, r'$\sigma_x$', va='center', ha='center', color='dimgray')


    if np.abs(sigma_2) > 1e-8:
        ax2.text(0.06, 0.3, r'$\sigma_2$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData, color='dodgerblue')
        ax2.text(0.06, -0.3, r'$\sigma_2$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData, color='dodgerblue')
    if np.abs(sigma_1) > 1e-8:
        ax2.text(0.35, -0.07, r'$\sigma_1$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData, color='navy')
        ax2.text(-0.35, -0.07, r'$\sigma_1$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData, color='navy')

    if np.abs(sigma_2) > 1e-8:
        ax2.arrow(0, 0.3, 0, 0.09 * (sigma_2 / 100), head_width=0.03, head_length=0.045 * np.abs(sigma_2 / 100), zorder=200, fc='dodgerblue', edgecolor='dodgerblue', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData)
        ax2.arrow(0, -0.3, 0, -0.09 * (sigma_2 / 100), head_width=0.03, head_length=0.045 * np.abs(sigma_2 / 100), zorder=200, fc='dodgerblue', edgecolor='dodgerblue', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData)

    if np.abs(sigma_1) > 1e-8:
        ax2.arrow(0.22, 0, 0.09 * (sigma_1 / 100), 0, head_width=0.03, head_length=0.045 * np.abs(sigma_1 / 100), zorder=200, fc='navy', edgecolor='navy', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData)
        ax2.arrow(-0.22, 0, -0.09 * (sigma_1 / 100), 0, head_width=0.03, head_length=0.045 * np.abs(sigma_1 / 100), zorder=200, fc='navy', edgecolor='navy', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), -theta) + ax2.transData)

    if np.abs(sigma_x) > 1e-8:
        ax2.arrow(0.55, 0, 0.09 * (sigma_x / 100), 0, head_width=0.03, head_length=0.045 * np.abs(sigma_x / 100), zorder=200, fc='dimgray', edgecolor='dimgray', width=0.005,
                 )
        ax2.arrow(-0.55, 0, -0.09 * (sigma_x / 100), 0, head_width=0.03, head_length=0.045 * np.abs(sigma_x / 100), zorder=200, fc='dimgray', edgecolor='dimgray', width=0.005,
                 )

    if np.abs(sigma_y) > 1e-8:
        ax2.arrow(0, 0.65, 0, 0.09 * (sigma_y / 100), head_width=0.03, head_length=0.045 * np.abs(sigma_y / 100), zorder=200, fc='dimgray', edgecolor='dimgray', width=0.005,
                 )
        ax2.arrow(0, -0.65, 0, -0.09 * (sigma_y/ 100), head_width=0.03, head_length=0.045 * np.abs(sigma_y / 100), zorder=200, fc='dimgray', edgecolor='dimgray', width=0.005,
                 )

    if np.abs(tau_xy) / tau_xy_max > 1e-8:
        ax2.text(0.2 *0.8/0.3 * 0.9, 0.12 * 0.8/0.3 / 2, r'$\tau_{xy}$', va='center', ha='center', )
        ax2.text(-0.11 *0.8/0.3 / 2 , -0.21 * 0.8/0.3 * 0.9, r'$\tau_{yx}$', va='center', ha='center', )
        ax2.text(-0.2 *0.8/0.3 * 0.9, -0.11 * 0.8/0.3 / 2 , r'$\tau_{xy}$', va='center', ha='center', )
        ax2.text(0.11 *0.8/0.3 / 2 , 0.21 * 0.8/0.3 * 0.9, r'$\tau_{yx}$', va='center', ha='center', )

    if np.abs(tau_xy) > 1e-8:
        ax2.arrow(0.18 * 0.8/0.3, -0.15 *0.8/0.3  * tau_xy / tau_xy_max / 8, 0, 0.8/0.3 * (0.3 - 0.045) * tau_xy / tau_xy_max / 8, head_width=0.03, head_length=0.045 * np.abs(tau_xy) / tau_xy_max / 2, zorder=200, fc='k', edgecolor='k', width=0.005,
                 )
        ax2.arrow(-0.18 * 0.8/0.3, 0.15 *0.8/0.3  * tau_xy / tau_xy_max / 8, 0, 0.8/0.3 * (-0.3 + 0.045) * tau_xy / tau_xy_max / 8, head_width=0.03, head_length=0.045 * np.abs(tau_xy) / tau_xy_max / 2, zorder=200, fc='k', edgecolor='k', width=0.005,
                 )
        ax2.arrow(-0.15 * 0.8/0.3 * tau_xy / tau_xy_max / 8, 0.18 * 0.8/0.3, 0.8/0.3 * (0.3 - 0.045)* tau_xy / tau_xy_max / 8, 0, head_width=0.03, head_length=0.045 * np.abs(tau_xy) / tau_xy_max / 2, zorder=200, fc='k', edgecolor='k', width=0.005,
                 )
        ax2.arrow(0.15 * 0.8/0.3 * tau_xy / tau_xy_max / 8, -0.18 * 0.8/0.3, 0.8/0.3 * (-0.3 + 0.045)* tau_xy / tau_xy_max / 8, 0, head_width=0.03, head_length=0.045 * np.abs(tau_xy) / tau_xy_max / 2, zorder=200, fc='k', edgecolor='k', width=0.005,
                 )

    square = mpl.patches.Rectangle((-0.4, -0.4), 0.8, 0.8, edgecolor='k', facecolor='none', zorder=1000)
    axs[0].add_patch(square)
    axs[0].axis('off')
    square_1 = mpl.patches.Rectangle((-0.7, -0.985), 1.415, 2.275, edgecolor='k', facecolor='none', zorder=1000,  clip_on=False)
    axs[0].add_patch(square_1)
    axs[0].text(-0.6, 1.185, r'a', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))
    l = 1
    xs = np.linspace(- l / 2, l / 2, 100)
    axs[0].plot(np.linspace(-0.4, 0.4, 100), np.zeros(100), c='k', lw=0.5, ls='-', zorder=-1)
    axs[0].plot(xs, xs * np.tan(np.deg2rad(-theta)), c='orchid', lw=2, ls=':', zorder=-1)
    axs[0].plot(xs, xs * np.tan(np.deg2rad(-theta) + np.pi/2), c='orchid', lw=2, ls=':', zorder=-1)
    axs[0].text(-0.5, 1.15, r'$\sigma_{1}= \ $'+f'{np.round(sigma_1, 2):5.4}MPa', color='navy')
    axs[0].text(-0.5, 1.05, r'$\sigma_{2}= \ $'+f'{np.round(sigma_2, 2):5.4}MPa', color='dodgerblue')


    axs[0].annotate('', xy=(-0.3, -0.79), xytext=(-0.6, -0.79),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4, headlength=3), zorder=100000, clip_on=False)
    axs[0].annotate('', xy=(-0.6, -0.49), xytext=(-0.6, -0.79),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4, headlength=3), zorder=100, clip_on=False)

    axs[0].plot(-0.6, -0.79, 'o', c='k', markersize=4, clip_on=False)
    axs[0].text(-0.275, -0.79, '$x$', fontsize=12, ha='center', va='center', zorder=100)
    axs[0].text(-0.65, -0.49, '$y$', fontsize=12, ha='center', va='center', zorder=100)

    wed = mpl.patches.Wedge((0, 0), 0.2, np.rad2deg(0), 180-theta, ec='k', fc='yellow', zorder=1000)
    axs[0].add_patch(wed)
    axs[0].text(0.1 * np.cos(np.deg2rad((180 - theta) / 2)), 0.1 * np.sin(np.deg2rad((180 - theta) / 2)), r'$\theta = $' + f'{np.round(180-theta, 1):.5}°',
             bbox=dict(boxstyle="round", fc="w", alpha=0.8), zorder=1e8, fontsize=7)



    square_2 = mpl.patches.Rectangle((-275, -225), 530, 520, edgecolor='k', facecolor='none', zorder=1000,  clip_on=False)
    axs[1].add_patch(square_2)
    plt.grid(ls=':', zorder=-1)
    axs[1].text(-250, 270, r'b', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8), clip_on=False)

    draw_rotated_square(axs[1], (intersect_x, intersect_y), 20, angle)  # 20 is the side length of the square
    draw_colored_rotated_square(axs[1], (intersect_x, intersect_y), 20, -angle)  # 20 is the side length of the square

    # Set axis limits
    sig_max_plot = 200
    sig_min_plot = -200
    mohr_axis(axs[1], sig_min_plot, sig_max_plot, 50)
    axs[1].set_xlim(sig_min_plot, sig_max_plot)
    axs[1].set_ylim(sig_min_plot, sig_max_plot)
    plt.show()

sigma_x_slider = widgets.IntSlider(description=r'$\sigma_x$',
                             value=100,
                             min=-100,
                             max=100,
                             step=1,
                             continuous_update=False)

sigma_y_slider = widgets.IntSlider(description=r'$\sigma_y$',
                             value=-100,
                             min=-100,
                             max=100,
                             step=1,
                             continuous_update=False)

tau_xy_slider = widgets.IntSlider(description=r'$\tau_{xy}$',
                             value=50,
                             min=0,
                             max=100,
                             step=1,
                             continuous_update=False)

interactive_plot = widgets.interactive(mohr_circle, sigma_x=sigma_x_slider, sigma_y=sigma_y_slider, tau_xy=tau_xy_slider)

# theta_slider.layout = widgets.Layout(width='445px')
interactive_plot.update()

output = widgets.VBox([interactive_plot.children[-1],
                       sigma_x_slider,
                       sigma_y_slider,
                       tau_xy_slider])
output.layout = widgets.Layout(display='flex',
                               flex_flow='column',
                               align_items='center',
                               align_content='center',
                               justify_content='center',
                               width='100%',
                               height='100%')
output